# 🌳Discovery of Previously Unknown **Geoglyphs** and Settlement Patterns in Acre, Brazil 

This nodebook  log for the **OpenAI to Z Challenge** , My Task is To identify and document previously unknown geoglyphs, village remnants, or other archaeological features in the Acre region using AI-assisted analysis of open-source lidar, satellite imagery, and historical records.

---

##  Data
| Layer | Source / DOI | Why It Matters |
|-------|--------------|----------------|
| **LIDAR Data** | 	OpenTopography | LIDAR can penetrate forest canopy, revealing hidden structures like earthworks, geoglyphs, and ancient roadways. |
| **Satellite Imagery** | Sentinel-2 Scenes | Multispectral imagery helps detect soil and vegetation anomalies indicative of human activity. |
| **Historical Records** | Biodiversity Heritage Library, Internet Archive, SciELO | Provide context on early explorations, indigenous settlements, and previously documented sites for comparative analysis.
|

---

# process 

To systematically discover unknown geoglyphs and settlement patterns in the Acre region, the following high-resolution data acquisition and AI-assisted processing steps were employed


A. LIDAR Data Acquisition
Access OpenTopography (https://opentopography.org) and download the LIDAR point cloud tiles covering Acre, Brazil.

Focus on Digital Elevation Models (DEMs) and Canopy Height Models (CHMs) to penetrate vegetation and reveal hidden surface anomalies.

B. Satellite Imagery (Sentinel-2)
Utilize Sentinel Playground (https://apps.sentinel-hub.com/sentinel-playground) to download recent cloud-free Sentinel-2 images (e.g., S2A_MSIL2A_20250610T143901...).

Prefer L2A products for surface reflectance values ideal for vegetation indices.

C. Historical & Archaeological Text Data
Scrape and parse structured texts from digital sources like the Biodiversity Heritage Library, Internet Archive, and SciELO for contextual mapping.




## Data  cell

In [ ]:
data = {
    "amazon_data_csv": "/kaggle/input/amazon/amazon.csv",
    "geojson_file": "/kaggle/input/amazon/amazon_boundary.geojson",
    "elevation_tif": "/kaggle/input/amazon/amazon_elevation.tif",
    "river_shapefile": {
        "shp": "/kaggle/input/amazon/amazon_river.shp",
        "shx": "/kaggle/input/amazon/amazon_river.shx",
        "dbf": "/kaggle/input/amazon/amazon_river.dbf"
    },
    "forest_cover_raster": "/kaggle/input/amazon/amazon_forest_cover.tif",
    "metadata_json": "/kaggle/input/amazon/metadata.json"
}


In [ ]:
!pip install earthengine-api --quiet
!pip install numpy pandas geopandas matplotlib rasterio shapely pyproj contextily earthengine-api --quiet


## ⚙️ Implementation

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
import contextily as cx
from shapely.geometry import Point, box
import ee  
from shapely.geometry import Point
from pyproj import CRS


## Define Acre, Brazil Bounding Box

In [ ]:
min_lon, min_lat = -74.0, -11.0  
max_lon, max_lat = -66.0, -7.0   


acre_bbox = box(min_lon, min_lat, max_lon, max_lat)


acre_gdf = gpd.GeoDataFrame({'region': ['Acre']}, geometry=[acre_bbox], crs="EPSG:4326")

## River Map

In [ ]:
ivers = gpd.read_file("HydroRIVERS_v10_sa_shp/HydroRIVERS_v10_sa.shp")
rivers = rivers.to_crs(epsg=3857)

world = gpd.read_file("natural_earth//ne_110m_admin_0_countries.shp")
south_america = world[world['CONTINENT'] == 'South America'].to_crs(epsg=3857)

fig, ax = plt.subplots(figsize=(15, 12), facecolor="#f8f9fa")

rivers.plot(ax=ax, color='royalblue', linewidth=0.5, alpha=0.7, label="Rivers")
south_america.boundary.plot(ax=ax, edgecolor='black', linewidth=1, label="Country Borders")

ax.set_xlim(rivers.total_bounds[0], rivers.total_bounds[2])
ax.set_ylim(rivers.total_bounds[1], rivers.total_bounds[3])

cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, zoom=5)

ax.set_title("South America River Network", fontsize=18, fontweight="bold", pad=15)
ax.set_facecolor("#eaeaea")
ax.grid(True, linestyle="--", linewidth=0.5, alpha=0.5)
ax.axis("off")

plt.tight_layout()
plt.show()


## Mound Villages and River Network in the Amazon

In [ ]:

rivers = gpd.read_file("/kaggle/input/amazon/amazon_river.shp").to_crs(epsg=3857)

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
south_america = world[world["continent"] == "South America"].to_crs(epsg=3857)

mound_df = pd.read_csv("/kaggle/input/amazon/amazon.csv")
mound_df = mound_df.rename(columns={"UTM X (Easting)": "x", "UTM Y (Northing)": "y"})

mound_gdf = gpd.GeoDataFrame(
    mound_df,
    geometry=gpd.points_from_xy(mound_df["x"], mound_df["y"]),
    crs="EPSG:32719"
).to_crs(epsg=3857)

fig, ax = plt.subplots(figsize=(15, 12), facecolor="#f8f9fa")

rivers.plot(ax=ax, color='royalblue', linewidth=0.5, alpha=0.5, label="Rivers")
south_america.boundary.plot(ax=ax, edgecolor='black', linewidth=1)
mound_gdf.plot(ax=ax, color='green', markersize=30, label='Mound Villages')

ax.set_xlim(rivers.total_bounds[0], rivers.total_bounds[2])
ax.set_ylim(rivers.total_bounds[1], rivers.total_bounds[3])

cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, zoom=5)

ax.set_title("Mound Villages and River Network in the Amazon", fontsize=18, fontweight='bold')
ax.axis("off")
plt.legend(loc='lower left')
plt.tight_layout()
plt.show()

## Sample 

In [ ]:
from IPython.display import Image, display

image_paths = [
    "/kaggle/input/sample/Screenshot 2025-06-26 192621.png",
    "/kaggle/input/sample/Screenshot 2025-06-26 193032.png",
    "/kaggle/input/sample/Screenshot 2025-06-26 192652.png"
]

for path in image_paths:
    display(Image(filename=path))


## Geoglyphs 

In [ ]:
geojson_url = "https://raw.githubusercontent.com/giswqs/data/main/amazon_geoglyphs.geojson"

geoglyphs = gpd.read_file(geojson_url).to_crs(epsg=3857)

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
south_america = world[world["continent"] == "South America"].to_crs(epsg=3857)

fig, ax = plt.subplots(figsize=(12, 10))
south_america.boundary.plot(ax=ax, edgecolor='black')
geoglyphs.plot(ax=ax, color='red', markersize=25, label="Geoglyphs")

cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, zoom=5)

ax.set_title("Amazon Geoglyphs Map", fontsize=16)
ax.axis("off")
plt.legend()
plt.tight_layout()
plt.show()

## Geolocation Details

In [ ]:
df = pd.read_csv("/kaggle/input/amazon/amazon.csv")

df.columns = [col.strip().lower() for col in df.columns]

latitude_col = 'latitude'
longitude_col = 'longitude'
area_col = 'area'

print("Geolocation Information:\n")

for index, row in df.iterrows():
    lat = row.get(latitude_col)
    lon = row.get(longitude_col)
    area = row.get(area_col)
    print(f"Feature {index + 1}: Latitude: {lat}, Longitude: {lon}, Area: {area}")